# rcsbsearch

Access the RCSB advanced search from python: [rcsbsearch.readthedocs.io](https://rcsbsearch.readthedocs.io)

    pip install rcsbsearch
    
## Demo

We are interested in how the antiviral drug boceprevir interacts with Covid-19. 
- Source Organism is "COVID-19"
- Structure title contains "protease"
- Bound to ligand "Boceprevir"

[RCSB Query](http://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entity_source_organism.taxonomy_lineage.name%22%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22COVID-19%22%2C%22negation%22%3Afalse%7D%2C%22node_id%22%3A0%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22value%22%3A%22protease%22%2C%22negation%22%3Afalse%7D%2C%22node_id%22%3A1%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22chem_comp.name%22%2C%22operator%22%3A%22contains_words%22%2C%22value%22%3A%22Boceprevir%22%2C%22negation%22%3Afalse%7D%2C%22node_id%22%3A2%7D%5D%7D%2C%22return_type%22%3A%22entry%22%2C%22request_info%22%3A%7B%22query_id%22%3A%2270e677a6376b4c5eba8b4f2b73866c92%22%2C%22src%22%3A%22ui%22%7D%7D)

In [1]:
from rcsbsearch import rcsb_attributes as attrs, TextQuery
import nglview

## Operator syntax
- Uses python comparison operators for basic attributes (`==`, `<`, `<=`, etc)
- Combine using set operators (`&`, `|`, `~`, etc)
- Execute queries as functions

In [2]:
q1 = attrs.rcsb_entity_source_organism.taxonomy_lineage.name == "COVID-19"
q2 = TextQuery("protease")
q3 = attrs.chem_comp.name.contains_words("Boceprevir")
q4 = attrs.rcsb_entry_info.resolution_combined > 1.5
query = q1 & q2 & q3 & ~q4

list(query())

['6WNP', '7K40']

In [3]:
nglview.show_pdbid('7brp')

NGLWidget()

## Fluent syntax

A second syntax is available with a [fluent interface](https://en.wikipedia.org/wiki/Fluent_interface), similar to popular data science packages like tidyverse and Apache Spark. Function calls  are chained together.

Here's an example around a second antiviral, remdesivir. The drug interferes with RNA polymerase, replacing an adenine and causing early chain termination. When integrated into RNA, the nucleotide formed from remdesivir has residue code F86.

In [4]:
attrs.struct.title.contains_phrase("RNA polymerase")\
    .or_(attrs.struct.title).contains_words("RdRp")\
    .and_(attrs.rcsb_chem_comp_container_identifiers.comp_id).exact_match("F86")\
    .and_(attrs.rcsb_entity_source_organism.taxonomy_lineage.name).exact_match("COVID-19")\
    .exec()\
    .iquery()

100%|██████████| 1/1 [00:00<?, ?it/s]


['7L1F', '7C2K', '7B3C', '7B3B']

In [5]:
view = nglview.show_pdbid('7B3C', default_representation=False)
#view.get_state()['_camera_orientation']
o = [6, 3, 23, 0, 23, 1, -6, 0, -2, 24, -2, 0, -84, -92, -109, 1]
view.control.orient(o)
view.add_surface(sele="protein", opacity=.8, color="electrostatic")
view.add_cartoon(sele="rna", color="cyan")
view.add_licorice(sele="rna", color="cyan")
view.add_spacefill(sele="F86")
view

NGLWidget()

## Try it!

[rcsbsearch.readthedocs.io](rcsbsearch.readthedocs.io)